In [1]:
import os

os.environ["SEED"] = "0"
import numpy as np
import pandas as pd
from ConfigSpace import Configuration

from src.aac.AAC import AAC
from src.configuration_space.LKH import CONFIGURATION_SPACE
from src.constant import DATA_DIR, SEED
from src.database import DB
from src.database.queries import *
from src.instance.Instance import Instance
from src.instance.InstanceList import InstanceList
from src.instance.TSP_Instance import TSP_Instance, TSP_train_test_from_index_file
from src.solver.Portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
from src.aac.SurrogateEstimator import Estimator1, SurrogateEstimator

In [2]:
train_instances, test_instances = TSP_train_test_from_index_file(
    filepath=DATA_DIR / "TSP" / "CEPS_benchmark" / "index.json",
    train_size=3,
)
train_instances

InstanceList(size=3)[TSP_Instance(filepath=TSP/CEPS_benchmark/explosion/21.tsp), TSP_Instance(filepath=TSP/CEPS_benchmark/rotation/34.tsp), TSP_Instance(filepath=TSP/CEPS_benchmark/compression/09.tsp)]

In [3]:
portfolio = Portfolio.from_solver_class(TSP_LKH_Solver, size=1)
portfolio

Portfolio(size=1)[Solver(id=52980316141715150)]

In [4]:
aac = AAC(
    portfolio=portfolio,
    instance_list=train_instances,
    prefix="config",
    t_c=600,
    calculate_features=False,
    estimator=None,
)

In [ ]:
for _ in aac.configure_iter():
    print(aac.get_progress())

In [ ]:
last_model_iter = 0

estimator = None
db = DB()
estimator_pct = 0.5
for _ in aac.configure_iter():
    if aac.get_progress() >= 0.5 and aac.iter - last_model_iter >= 10:
        X, y = get_model_training_data(db)
        estimator = Estimator1(max_cost=TSP_LKH_Solver.MAX_COST, estimator_pct=estimator_pct)
        estimator.fit(X, y)
        estimator.log()
        last_model_iter = aac.iter
        aac.update(estimator=estimator)

In [2]:
# db = DB()
db = DB("database/run-test_924008.db")

In [3]:
pd.read_sql_query("SELECT * FROM instances", db._conn)

,id,filepath,optimum,angle_min,angle_median,angle_mean,angle_max,angle_sd,angle_span,angle_coef_of_var,...,bc_no1s_min,bc_no1s_q25,bc_no1s_q50,bc_no1s_q75,bc_no1s_max,bc_p1s,bc_pn1s,bcTime,acc,acfTime
0,855157651668393360,TSP/CEPS_benchmark/explosion/21.tsp,14928292.0,0.006290,1.484946,1.522267,3.139028,0.916031,3.132738,0.601754,...,0.125000,0.500000,0.5,0.500000,0.875000,0.90125,0.20000,0.21,448.283903,0.68
1,1353678304570487518,TSP/CEPS_benchmark/rotation/34.tsp,15486460.0,0.000684,1.495912,1.523094,3.138384,0.912371,3.137700,0.599025,...,0.111111,0.500000,0.5,0.500000,0.888889,0.87750,0.24750,0.08,400.212239,0.42
2,810870798045289410,TSP/CEPS_benchmark/compression/09.tsp,16003060.0,0.000000,1.417332,1.483901,3.141593,1.026220,3.141593,0.691569,...,0.166667,0.500000,0.5,0.500000,0.833333,0.87125,0.26500,0.10,361.985346,0.71
3,1306299629087479721,TSP/CEPS_benchmark/implosion/34.tsp,19968838.0,0.000395,1.483846,1.518058,3.138849,0.916994,3.138454,0.604057,...,0.125000,0.333333,0.5,0.666667,0.875000,0.86500,0.27375,0.10,383.613467,0.56
4,410041616729589825,TSP/CEPS_benchmark/linearprojection/25.tsp,13578491.0,0.009872,1.625140,1.570276,3.131472,0.931936,3.121600,0.593485,...,0.250000,0.500000,0.5,0.500000,0.750000,0.87500,0.25500,0.15,362.656335,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,707351994282137341,TSP/CEPS_benchmark/cluster/19.tsp,19287084.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,540324668286432217,TSP/CEPS_benchmark/uniform_portgen/27.tsp,20825913.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,1765803814609198146,TSP/CEPS_benchmark/grid/00.tsp,21258773.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,1378924496111515287,TSP/CEPS_benchmark/explosion/32.tsp,12927140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
pd.read_sql_query("SELECT * FROM solvers", db._conn)

,id,ASCENT_CANDIDATES,BACKBONE_TRIALS,BACKTRACKING,CANDIDATE_SET_TYPE,EXTRA_CANDIDATES,EXTRA_CANDIDATE_SET_TYPE,GAIN23,GAIN_CRITERION,INITIAL_STEP_SIZE,...,MAX_BREADTH,MAX_CANDIDATES,MOVE_TYPE,PATCHING_A,PATCHING_C,POPULATION_SIZE,RESTRICTED_SEARCH,SUBGRADIENT,SUBSEQUENT_MOVE_TYPE,SUBSEQUENT_PATCHING
0,52980316141715150,0.60,1.0,0.0,0.0,0.7,0.0,1.0,0.0,0.00,...,0.444643,1.000000,0.25,0.6,0.2,0.061224,0.0,0.0,5.0,1.0
1,287011037061967493,0.75,0.0,0.0,0.0,0.3,0.0,1.0,0.0,0.00,...,0.592453,0.777778,0.75,0.8,0.2,0.836735,1.0,1.0,3.0,1.0
2,176228367859327934,0.85,1.0,0.0,0.0,0.4,0.0,0.0,1.0,0.50,...,0.354664,0.333333,0.25,0.8,0.4,0.408163,1.0,1.0,2.0,0.0
3,2041120758543581287,0.80,1.0,0.0,2.0,0.7,0.0,1.0,0.0,0.25,...,0.365942,1.000000,0.25,0.6,0.8,0.051020,0.0,1.0,5.0,0.0
4,1056464823932360013,0.65,1.0,1.0,2.0,0.0,0.0,0.0,1.0,1.00,...,0.322513,0.666667,1.00,0.8,0.8,0.112245,0.0,1.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,230211342664630695,0.45,0.0,1.0,3.0,1.0,0.0,1.0,0.0,0.25,...,0.277844,0.777778,0.25,0.0,0.4,0.989796,1.0,1.0,2.0,1.0
195,756183311605343287,0.75,0.0,1.0,3.0,0.6,0.0,1.0,0.0,0.00,...,0.356703,0.777778,1.00,0.4,0.4,0.326531,1.0,0.0,2.0,1.0
196,1925419895121924088,0.30,1.0,0.0,2.0,0.7,0.0,0.0,1.0,0.00,...,0.402047,0.000000,1.00,0.8,0.8,0.244898,1.0,0.0,4.0,1.0
197,670624953675152897,0.50,0.0,1.0,3.0,0.7,0.0,0.0,1.0,0.50,...,0.361161,0.777778,0.00,0.4,0.2,0.479592,1.0,1.0,4.0,0.0


In [5]:
results = pd.read_sql_query("SELECT * FROM results", db._conn)
results

,id,prefix,solver_id,instance_id,cost,time,cached,surrogate,error
0,config;aac_iter=1_52980316141715150_8551576516...,config;aac_iter=1,52980316141715150,855157651668393360,100.00,22.368366,0,0,0
1,config;aac_iter=1_52980316141715150_1353678304...,config;aac_iter=1,52980316141715150,1353678304570487518,100.00,16.106277,0,0,0
2,config;aac_iter=1_52980316141715150_1306299629...,config;aac_iter=1,52980316141715150,1306299629087479721,100.00,15.510598,0,0,0
3,config;aac_iter=1_52980316141715150_8108707980...,config;aac_iter=1,52980316141715150,810870798045289410,100.00,16.264867,0,0,0
4,config;aac_iter=1_52980316141715150_4100416167...,config;aac_iter=1,52980316141715150,410041616729589825,100.00,17.098791,0,0,0
...,...,...,...,...,...,...,...,...,...
1085,test_998200510323771390_366756074013316631,test,998200510323771390,366756074013316631,0.08,0.080000,0,0,0
1086,test_998200510323771390_1905434050073119983,test,998200510323771390,1905434050073119983,100.00,10.000000,0,0,0
1087,test_998200510323771390_1765803814609198146,test,998200510323771390,1765803814609198146,4.36,4.360000,0,0,0
1088,test_998200510323771390_707351994282137341,test,998200510323771390,707351994282137341,6.54,6.540000,0,0,0


In [6]:
_ = results.loc[results["prefix"].str.startswith("config")]
(_.loc[lambda x: x["surrogate"] == 0, "cost"] < 100).value_counts()

False    455
True     215
Name: cost, dtype: int64

In [ ]:
pd.read_sql_query("SELECT * FROM evaluations", db._conn)